In [ ]:
%pip install hvac

In [ ]:
import hvac
import matplotlib.pyplot as plt
import os
import requests

In [ ]:
client = hvac.Client(
    url=os.getenv('VAULT_API'),
    token=os.getenv('VAULT_TOKEN'),
)

print(client.is_authenticated())

In [ ]:
PATH='docker-fairbanksio'
#PATH='docker-jonfairbanks'

In [ ]:
username=None
access_token=None

In [ ]:
try:
    secret_resp = client.secrets.kv.v2.read_secret_version(
        mount_point='kv', 
        path=PATH, 
        raise_on_deleted_version=False
    )
    
    if secret_resp['data'] is not None:
        secret_value = secret_resp['data']['data']
        username = secret_value['DOCKER_USERNAME']
        access_token = secret_value['DOCKER_ACCESS_TOKEN_TEST']
    else:
        print("The secret does not exist.")
except hvac.exceptions.InvalidPath:
    print("The path is invalid or the permission is denied.")
except hvac.exceptions.Forbidden:
    print("The permission is denied.")
except hvac.exceptions.VaultError as e:
    print(f"Vault error occurred: {e}")

In [ ]:
# Get the total number of images
headers = {"Authorization": f"Bearer {access_token}"}
response = requests.get(
    f"https://hub.docker.com/v2/repositories/{username}/?page_size=1",
    headers=headers,
)
total_count = response.json()["count"]
print(f"Found {total_count} Docker images")

In [ ]:
# Loop through all pages of images
page_size = 100
all_images = []
for page in range(1, (total_count // page_size) + 2):
    response = requests.get(
        f"https://hub.docker.com/v2/repositories/{username}/?page_size={page_size}&page={page}",
        headers=headers,
    )
    data = response.json()["results"]
    all_images.extend(data)

# Sort the images based on the number of pulls
sorted_images = sorted(all_images, key=lambda x: x["pull_count"], reverse=True)

# Print the sorted images
for image in sorted_images:
    repo_name = image["name"]
    namespace = image["namespace"]
    repo_url = f"https://hub.docker.com/r/{namespace}/{repo_name}"
    repo_pulls = image["pull_count"]
    print(f"{namespace}/{repo_name} -- {repo_pulls:,} pulls")


In [ ]:
# Get data for top 10 images
top_10 = sorted_images[:10]
labels = [f'{image["namespace"]}/{image["name"]}' for image in top_10]
pull_counts = [image["pull_count"] for image in top_10]

In [ ]:
# Create horizontal bar chart
fig, ax = plt.subplots(figsize=(8, 5))
ax.barh(labels, pull_counts)
ax.set_xlabel("Number of Pulls")
ax.set_ylabel("Image")
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_title("Top 10 Most Downloaded Docker Images")
plt.tight_layout()
plt.show()